In [1]:
from nlb_tools.nwb_interface import NWBDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import math
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966 (90 deg)
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0 (0 deg)
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793 (180 deg)
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle

# If you would prefer to load the '.h5' example file rather than the '.pickle' example file. You need the deepdish package
# import deepdish as dd 

#Import function to get the covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

#Import decoder functions
from Neural_Decoding.decoders import WienerCascadeDecoder
from Neural_Decoding.decoders import WienerFilterDecoder
from Neural_Decoding.decoders import DenseNNDecoder
from Neural_Decoding.decoders import SimpleRNNDecoder
from Neural_Decoding.decoders import GRUDecoder
from Neural_Decoding.decoders import LSTMDecoder
from Neural_Decoding.decoders import XGBoostDecoder
from Neural_Decoding.decoders import SVRDecoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

def get_sses_pred(y_test,y_test_pred):
    sse=np.sum((y_test_pred-y_test)**2,axis=0)
    return sse

def get_sses_mean(y_test):
    y_mean=np.mean(y_test,axis=0)
    sse_mean=np.sum((y_test-y_mean)**2,axis=0)
    return sse_mean

def nans(shape, dtype=float):
    a = np.empty(shape, dtype)
    a.fill(np.nan)
    return a

def vector_reject(u,v):
    #project u on v, subtract u1 from u
    P = np.outer(v,(v.T))/(v@(v.T))
    u_sub = u - P@u
#     another calculation, to double-check
#     v_norm = np.sqrt(sum(v**2))    
#     proj_u_on_v = (np.dot(u, v)/v_norm**2)*v
#     u_sub = u - proj_u_on_v
    return u_sub

def calc_proj_matrix(A):
    return A@np.linalg.inv(A.T@A)@A.T
def calc_proj(b, A):
    P = calc_proj_matrix(A)
    return P@b.T

# Single Lag

In [2]:
foldername = "~/area2_population_analysis/s1-kinematics/actpas_NWB/"
monkey = "Lando_20170731"
filename = foldername + monkey + "_COactpas_TD.nwb"

dataset_5ms = NWBDataset(filename, split_heldout=False)
dataset_5ms.resample(5)
dataset_5ms.smooth_spk(40, name='smth_40')
bin_width = dataset_5ms.bin_width
print(bin_width)

5


In [3]:
n_dims = 20 # for PCA

active_mask = (~dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')
passive_mask = (dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')


trial_mask = active_mask
n_trials = dataset_5ms.trial_info.loc[trial_mask].shape[0]
print(n_trials,'trials')
n_neurons = dataset_5ms.data.spikes.shape[1]
print(n_neurons,'neurons')

all_data = np.array(dataset_5ms.data.spikes_smth_40)
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)

scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims)
X = pca.fit(X)

PCA_data = nans([all_data.shape[0],n_dims])
idx = 0
for dp in all_data:
    dp = dp.reshape((1, -1))
    if np.isnan(dp).any():
        dp_pca = nans([1,n_dims])
    else:
        dp_pca = pca.transform(scaler.transform(dp))
    PCA_data[idx,:] = dp_pca
    idx+=1
print(PCA_data.shape)
dataset_5ms.add_continuous_data(PCA_data,'PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

#make dictionary for trial condition (reaching directions) for Stratified CV
active_trials_idx = np.array(dataset_5ms.trial_info.loc[trial_mask]['trial_id'])
cond_dir_idx = []
cond_dict = nans([n_trials])
for direction in [0,45,90,135,180,225,270,315]:
    cond_dir_idx.append(np.where((dataset_5ms.trial_info['cond_dir'] == direction) & (dataset_5ms.trial_info['ctr_hold_bump'] == False) & \
           (dataset_5ms.trial_info['split'] != 'none'))[0])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(cond_dir_idx)):
        if idx in cond_dir_idx[cond]:
            cond_dict[i] = cond
            break
    i+=1
print(cond_dict)

371 trials
87 neurons
(538986, 87)
(538986, 87)
(538986, 20)
PCA total var explained: 0.4726979696371898
[0. 6. 6. 4. 6. 2. 0. 4. 6. 4. 6. 0. 4. 4. 4. 4. 4. 6. 4. 0. 6. 6. 0. 6.
 6. 2. 4. 6. 0. 2. 6. 4. 4. 0. 6. 6. 6. 0. 0. 2. 2. 6. 4. 4. 0. 6. 6. 6.
 6. 6. 2. 6. 6. 0. 2. 2. 2. 4. 6. 0. 2. 0. 0. 2. 0. 6. 2. 0. 0. 4. 6. 2.
 6. 6. 2. 0. 0. 6. 0. 6. 0. 4. 0. 4. 0. 4. 6. 6. 6. 0. 0. 6. 2. 0. 4. 0.
 4. 0. 2. 2. 2. 2. 4. 2. 0. 0. 0. 0. 6. 6. 0. 0. 4. 2. 6. 4. 0. 6. 2. 6.
 4. 0. 2. 4. 4. 4. 6. 4. 4. 0. 2. 4. 2. 2. 0. 0. 2. 4. 2. 2. 4. 0. 6. 2.
 4. 6. 6. 6. 4. 4. 6. 2. 6. 4. 6. 2. 4. 6. 2. 4. 6. 2. 0. 2. 6. 6. 2. 4.
 6. 6. 2. 6. 4. 6. 4. 2. 6. 0. 0. 0. 2. 0. 4. 0. 4. 0. 2. 4. 4. 2. 4. 2.
 4. 4. 6. 6. 4. 0. 4. 4. 4. 0. 0. 4. 0. 6. 4. 0. 2. 0. 0. 2. 0. 4. 0. 4.
 2. 2. 2. 2. 4. 0. 4. 4. 6. 4. 0. 4. 4. 0. 0. 6. 4. 2. 0. 4. 6. 6. 4. 0.
 6. 4. 4. 6. 2. 0. 2. 0. 2. 6. 2. 6. 2. 2. 0. 0. 4. 2. 4. 2. 2. 6. 0. 0.
 6. 6. 0. 0. 0. 0. 2. 2. 0. 0. 6. 0. 6. 4. 6. 6. 0. 6. 4. 2. 4. 2. 6. 2.
 6. 6. 0. 4. 6. 0. 

In [4]:
# # Prepare for plotting

# active_data = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=(-100, 500), ignored_trials=~trial_mask)
# active_trials_pca = nans([n_trials,n_timepoints,n_dims])
# i = 0
# for idx, trial in active_data.groupby('trial_id'):
#     active_trials_pca[i,:,:]=trial.PCA.to_numpy()
#     i+=1
# print(active_trials_pca.shape)

# plot_dir = np.array([0,45,90,135,180,225,270,315]) # limit plot directions to reduce cluttering
# directions = np.array([0,45,90,135,180,225,270,315])
# pred_range = (-100, 500)
# x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)

# # define some useful time points
# move_idx=0
# ret_idx = 200

# import matplotlib as mpl
# cmap = plt.get_cmap('coolwarm',len(plot_dir))
# custom_palette = [mpl.colors.rgb2hex(cmap(i)) for i in range(len(plot_dir))]

# plot_dims = 10

# fig,ax=plt.subplots(plot_dims,1,figsize=(10,20))
# for i in range(plot_dims):
#     for j in range(len(plot_dir)):
#         color = custom_palette[j]
#         dir_idx = np.argwhere(directions == plot_dir[j])[0]
#         cond_mean_proj = np.mean(active_trials_pca[np.argwhere(cond_dict==dir_idx).flatten(),:,:], axis = 0)[:,i] 
#         pca_mean = np.mean(active_data.PCA.to_numpy(),axis = 0)[i]
#         ax[i].plot(x_axis,cond_mean_proj - pca_mean,linewidth=2.25,color = color,label = plot_dir[j])
        
#         ax[i].axvline(move_idx, color='k',linewidth = .5)
#         ax[i].axvline(ret_idx, color='k',linewidth = .5)
# #         ax[i].set_xlim([0,T])
#         ax[i].set_ylim([-6, 6])
#         ax[i].axhline(0,color ='k',ls = '--')
#         if i<plot_dims-1:
#             ax[i].set_xticks([])
#         else:
#             ax[i].set_xlabel('Time (ms)')
            
#         ax[i].set_yticks([])
#         ax[i].set_ylabel('Dim. '+str(i+1))

#     ax[0].set_title('PCA Projections')
    
# plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')

In [5]:
def process_train_test(X,y,training_set,test_set):
    X_train = X[training_set,:,:]
    X_test = X[test_set,:,:]
    y_train = y[training_set,:,:]
    y_test = y[test_set,:,:]

    #flat by trials
    X_flat_train = X_train.reshape((X_train.shape[0]*X_train.shape[1]),X_train.shape[2])
    X_flat_test = X_test.reshape((X_test.shape[0]*X_test.shape[1]),X_test.shape[2])
    y_train=y_train.reshape((y_train.shape[0]*y_train.shape[1]),y_train.shape[2])
    y_test=y_test.reshape((y_test.shape[0]*y_test.shape[1]),y_test.shape[2])
    
    X_flat_train_mean=np.nanmean(X_flat_train,axis=0)
    X_flat_train_std=np.nanstd(X_flat_train,axis=0)   
    #array with only 0 will have 0 std and cause errors
    X_flat_train_std[X_flat_train_std==0] = 1
    
    X_flat_train=(X_flat_train-X_flat_train_mean)/X_flat_train_std
    X_flat_test=(X_flat_test-X_flat_train_mean)/X_flat_train_std
    y_train_mean=np.mean(y_train,axis=0)
    y_train=y_train-y_train_mean
    y_test=y_test-y_train_mean    
    
    return X_flat_train,X_flat_test,y_train,y_test

In [6]:
def fit_and_predict(dataset, trial_mask, align_field, align_range, lag, x_field, y_field):
    """Extracts spiking and kinematic data from selected trials and fits linear decoder"""
    # Extract rate data from selected trials
    vel_df = dataset.make_trial_data(align_field=align_field, align_range=align_range, ignored_trials=~trial_mask)
    # Lag alignment for kinematics and extract kinematics data from selected trials
    lag_align_range = (align_range[0] + lag, align_range[1] + lag)
    rates_df = dataset.make_trial_data(align_field=align_field, align_range=lag_align_range, ignored_trials=~trial_mask)
    
    n_trials = rates_df['trial_id'].nunique()
    n_timepoints = int((align_range[1] - align_range[0])/dataset.bin_width)
    n_neurons = rates_df[x_field].shape[1]
    
    lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 1, 6)})
    rates_array = rates_df[x_field].to_numpy()
    vel_array = vel_df[y_field].to_numpy()
    lr_all.fit(rates_array, vel_array)
    pred_vel = lr_all.predict(rates_array)
    vel_df = pd.concat([vel_df, pd.DataFrame(pred_vel, columns=dataset._make_midx('pred_vel', ['x', 'y'], 2))], axis=1)
#     print(lr_all.best_params_['alpha'])
    
    rates_array = rates_array.reshape(n_trials, n_timepoints, n_neurons)
    vel_array = vel_array.reshape(n_trials, n_timepoints, 2)
    
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
    true_concat = nans([n_trials*n_timepoints,2])
    pred_concat = nans([n_trials*n_timepoints,2])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = process_train_test(rates_array,vel_array,training_set,test_set)
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 1, 6)}) 
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)
        
        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n
    
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)     
    print('R2:',R2) 
    return R2, lr_all.best_estimator_.coef_, vel_df

In [7]:
def sub_and_predict(dataset, trial_mask, align_field, align_range, lag, x_field, y_field, weights):
    """Extracts spiking and kinematic data from selected trials and fits linear decoder"""
    # Extract rate data from selected trials
    vel_df = dataset.make_trial_data(align_field=align_field, align_range=align_range, ignored_trials=~trial_mask)
    # Lag alignment for kinematics and extract kinematics data from selected trials
    lag_align_range = (align_range[0] + lag, align_range[1] + lag)
    rates_df = dataset.make_trial_data(align_field=align_field, align_range=lag_align_range, ignored_trials=~trial_mask)
    
    n_trials = rates_df['trial_id'].nunique()
    n_timepoints = int((align_range[1] - align_range[0])/dataset.bin_width)
    n_neurons = rates_df[x_field].shape[1]

    rates_array = rates_df[x_field].to_numpy() - calc_proj(rates_df[x_field].to_numpy(),weights.T).T
    vel_array = vel_df[y_field].to_numpy()
    
    lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 1, 6)})
    lr_all.fit(rates_array, vel_array)
    pred_vel = lr_all.predict(rates_array)
    vel_df = pd.concat([vel_df, pd.DataFrame(pred_vel, columns=dataset._make_midx('pred_vel', ['x', 'y'], 2))], axis=1)
#     print(lr_all.best_params_['alpha'])
    
    rates_array = rates_array.reshape(n_trials, n_timepoints, n_neurons)
    vel_array = vel_array.reshape(n_trials, n_timepoints, 2)
    
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
    true_concat = nans([n_trials*n_timepoints,2])
    pred_concat = nans([n_trials*n_timepoints,2])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = process_train_test(rates_array,vel_array,training_set,test_set)
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 1, 6)}) 
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)
        
        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n
    
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)     
    print('R2:',R2) 
    return R2, lr_all.best_estimator_.coef_, vel_df

## with Neurons

In [8]:
lag_axis = np.arange(-300,300,20)
x_field = 'spikes_smth_40'
y_field ='hand_vel'
trial_mask = active_mask

# Prepare for plotting
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
plot_dim = 'x' # plot x velocity
colors = ['red', 'blue', 'green', 'orange']

figDir = "/Users/sherryan/area2_population_analysis/figures/neurons/act/"

dim = n_neurons

In [9]:
pred_range = (-100, 500)
label = '_whole_'
x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)

curr_r2_array = nans([len(lag_axis)])
curr_coef_array = nans([len(lag_axis),2,dim])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    curr_r2_array[i] = r2
    curr_coef_array[i,:,:] = coef

idx_max = np.argmax(curr_r2_array)
time_max = lag_axis[idx_max]
_, _, vel_df = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial.hand_vel[plot_dim], color=color, linewidth=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('x-vel')
plt.tight_layout()
plt.savefig(figDir + monkey + label + 'true.png', dpi = 'figure')
plt.close()

for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('x-vel')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'_pred.png', dpi = 'figure')
plt.close()

plt.plot(lag_axis, curr_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting hand velocity [-100,500]')
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'.png', dpi = 'figure')
plt.close()

# ang_dist_to_max = nans([len(lag_axis)])
# for i in range(0, len(curr_coef_array)):
#     ang_dist_to_max[i] = math.degrees(angle_between(curr_coef_array[i,0,:],curr_coef_array[idx_max,0,:]))
# plt.scatter(lag_axis, ang_dist_to_max)
# plt.title('Angular distance to X-vel decoding dim at t_max')
# plt.xlabel('Time lag (ms)')
# plt.ylabel('Angular distance (degrees)')
# plt.show()

weights = curr_coef_array[idx_max,:,:]
for iter in range(0,7):  
    #subtract predictions with primary decoding dimensions (at time with max R2)
    sub_coef_array = nans([len(lag_axis),2,dim])
    sub_r2_array = nans([len(lag_axis)])

    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,weights)
        sub_r2_array[i] = r2
        sub_coef_array[i,:,:] = coef

    plt.plot(lag_axis,sub_r2_array)
    plt.title('R2 score projecting out #'+ str(iter+1) +' t_max dim')
    idx_max = np.argmax(sub_r2_array)
    time_max = lag_axis[idx_max]
    plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
    plt.legend()
    plt.xlabel('Time lag (ms)')
    plt.ylabel('R2')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'.png', dpi = 'figure')
    plt.close()

    _, _, vel_df = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field,weights)
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel('x-vel')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'_pred.png', dpi = 'figure')
    plt.close()
#     plt.plot(lag_axis,np.subtract(sub_r2_array,curr_r2_array))
#     plt.axhline(0,color = 'k',linestyle='--')
#     plt.title('R2 difference after projecting out t_max dim')
#     plt.xlabel('Time lag (ms)')
#     plt.ylabel('R2 difference')
#     plt.show()
#     curr_r2_array = sub_r2_array

    #stack the decoding dimensions to be projected out
    weights = np.vstack((weights,sub_coef_array[idx_max,:,:]))

R2: 0.04487391562437748
R2: 0.06957057596127358
R2: 0.09804973776191162
R2: 0.1308959198200862
R2: 0.1691114312981591
R2: 0.21331173367960976
R2: 0.2626127954119539
R2: 0.31507136141787573
R2: 0.3683185149050926
R2: 0.42041304582226857
R2: 0.46989235143346164
R2: 0.5147226184887073
R2: 0.552611615874187
R2: 0.58171696310109
R2: 0.6015942818640538
R2: 0.6128427785002228
R2: 0.6160987909721423
R2: 0.6120028245611691
R2: 0.6017528693457472
R2: 0.5870101450328304
R2: 0.5697738351216108
R2: 0.5521672812537423
R2: 0.5361227200877473
R2: 0.5230898340263261
R2: 0.5133828972598026
R2: 0.5059570854972484
R2: 0.4991485943549392
R2: 0.4916334111249465
R2: 0.4827250041457416
R2: 0.47232650511179763
R2: 0.6160987909721423
R2: 0.03926321917679165
R2: 0.06080448887424339
R2: 0.08475324606850754
R2: 0.11152864551054864
R2: 0.1419252628608193
R2: 0.17621192510889228
R2: 0.21297270489330478
R2: 0.24919498008020324
R2: 0.2819024642061516
R2: 0.30988008239002574
R2: 0.3338784462739306
R2: 0.355682146043682

In [10]:
pred_range = (0, 120)
label = '_early_'
x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)

curr_r2_array = nans([len(lag_axis)])
curr_coef_array = nans([len(lag_axis),2,dim])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    curr_r2_array[i] = r2
    curr_coef_array[i,:,:] = coef

idx_max = np.argmax(curr_r2_array)
time_max = lag_axis[idx_max]
_, _, vel_df = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial.hand_vel[plot_dim], color=color, linewidth=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('x-vel')
plt.tight_layout()
plt.savefig(figDir + monkey + label + 'true.png', dpi = 'figure')
plt.close()

for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('x-vel')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'_pred.png', dpi = 'figure')
plt.close()

plt.plot(lag_axis, curr_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting hand velocity [0,120]')
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'.png', dpi = 'figure')
plt.close()

weights = curr_coef_array[idx_max,:,:]
for iter in range(0,3):  
    #subtract predictions with primary decoding dimensions (at time with max R2)
    sub_coef_array = nans([len(lag_axis),2,dim])
    sub_r2_array = nans([len(lag_axis)])

    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,weights)
        sub_r2_array[i] = r2
        sub_coef_array[i,:,:] = coef

    plt.plot(lag_axis,sub_r2_array)
    plt.title('R2 score projecting out #'+ str(iter+1) +' t_max dim')
    idx_max = np.argmax(sub_r2_array)
    time_max = lag_axis[idx_max]
    plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
    plt.legend()
    plt.xlabel('Time lag (ms)')
    plt.ylabel('R2')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'.png', dpi = 'figure')
    plt.close()


    _, _, vel_df = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field,weights)
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel('x-vel')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'_pred.png', dpi = 'figure')
    plt.close()

    #stack the decoding dimensions to be projected out
    weights = np.vstack((weights,sub_coef_array[idx_max,:,:]))

R2: -0.17144769637932256
R2: -0.18960197011496294
R2: -0.23853637136404315
R2: -0.30057508443337255
R2: -0.32038765265819347
R2: -0.2714794605090982
R2: -0.23754155734290228
R2: -0.20794353090780637
R2: -0.11992783329039058
R2: 0.0430098873603495
R2: 0.2266010901957053
R2: 0.38585624269844343
R2: 0.4924175203679172
R2: 0.5561060924519965
R2: 0.5892108339427159
R2: 0.5955604440279927
R2: 0.5893925703277675
R2: 0.5849634701395976
R2: 0.5789492247957497
R2: 0.568057965889406
R2: 0.5687450947291666
R2: 0.5674674788466568
R2: 0.5599339334533522
R2: 0.5460269616607429
R2: 0.5254830062941698
R2: 0.5068300094067473
R2: 0.4938975192816373
R2: 0.4888143978869912
R2: 0.3819459469337636
R2: 0.40291896115888914
R2: 0.5955604440279927
R2: -0.1668270924685018
R2: -0.18380863725126573
R2: -0.22443543209737427
R2: -0.2689755659854016
R2: -0.2813595839021412
R2: -0.2312747319590125
R2: -0.19291200327099878
R2: -0.18574629449112434
R2: -0.13060564337845637
R2: 0.012044650516197741
R2: 0.17048006477313538

In [11]:
pred_range = (380, 500)
label = '_late_'
x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)

curr_r2_array = nans([len(lag_axis)])
curr_coef_array = nans([len(lag_axis),2,dim])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    curr_r2_array[i] = r2
    curr_coef_array[i,:,:] = coef

idx_max = np.argmax(curr_r2_array)
time_max = lag_axis[idx_max]
_, _, vel_df = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial.hand_vel[plot_dim], color=color, linewidth=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('x-vel')
plt.tight_layout()
plt.savefig(figDir + monkey + label + 'true.png', dpi = 'figure')
plt.close()

for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('x-vel')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'_pred.png', dpi = 'figure')
plt.close()

plt.plot(lag_axis, curr_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting hand velocity [380,500]')
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'.png', dpi = 'figure')
plt.close()

weights = curr_coef_array[idx_max,:,:]
for iter in range(0,3):  
    #subtract predictions with primary decoding dimensions (at time with max R2)
    sub_coef_array = nans([len(lag_axis),2,dim])
    sub_r2_array = nans([len(lag_axis)])

    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,weights)
        sub_r2_array[i] = r2
        sub_coef_array[i,:,:] = coef

    plt.plot(lag_axis,sub_r2_array)
    plt.title('R2 score projecting out #'+ str(iter+1) +' t_max dim')
    idx_max = np.argmax(sub_r2_array)
    time_max = lag_axis[idx_max]
    plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
    plt.legend()
    plt.xlabel('Time lag (ms)')
    plt.ylabel('R2')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'.png', dpi = 'figure')
    plt.close()


    _, _, vel_df = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field,weights)
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel('x-vel')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'_pred.png', dpi = 'figure')
    plt.close()

    #stack the decoding dimensions to be projected out
    weights = np.vstack((weights,sub_coef_array[idx_max,:,:]))

R2: 0.38525820554664825
R2: 0.38707635252648176
R2: 0.3773383281831364
R2: 0.37799470134855195
R2: 0.39339459463002147
R2: 0.4149754764297644
R2: 0.44001422323045036
R2: 0.46868852038927256
R2: 0.49721458469882573
R2: 0.4749919393654032
R2: 0.46082424125104116
R2: 0.5031003362197723
R2: 0.5428885898690555
R2: 0.5923360707010129
R2: 0.6097290537907152
R2: 0.6191041802286824
R2: 0.6166095401935408
R2: 0.606164189337595
R2: 0.5977415386302625
R2: 0.5916230506025818
R2: 0.5803987418668325
R2: 0.5614601360418886
R2: 0.5321161559970004
R2: 0.49676397389687377
R2: 0.463046741407105
R2: 0.4296449160305177
R2: 0.3994392260130196
R2: 0.37714947222273265
R2: 0.3542175224163765
R2: 0.3262207783886649
R2: 0.6191041802286824
R2: 0.38928971124597245
R2: 0.38323731071795897
R2: 0.3617770932330153
R2: 0.3565074118103265
R2: 0.37877604224304173
R2: 0.40927886900665134
R2: 0.437337170263263
R2: 0.4574568744173221
R2: 0.4485005496556732
R2: 0.38722296986579907
R2: 0.2935024448063456
R2: 0.2437275107792078

## with PCA

In [12]:
x_field = 'PCA'
y_field ='hand_vel'
lag_axis = np.arange(-300,300,20)

figDir = "/Users/sherryan/area2_population_analysis/figures/PCA/act/"

dim = n_dims

In [13]:
pred_range = (-100, 500)
label = '_whole_'
x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)

curr_r2_array = nans([len(lag_axis)])
curr_coef_array = nans([len(lag_axis),2,dim])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    curr_r2_array[i] = r2
    curr_coef_array[i,:,:] = coef

idx_max = np.argmax(curr_r2_array)
time_max = lag_axis[idx_max]
_, _, vel_df = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('x-vel')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'_pred.png', dpi = 'figure')
plt.close()

plt.plot(lag_axis, curr_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting hand velocity [-100,500]')
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'.png', dpi = 'figure')
plt.close()

weights = curr_coef_array[idx_max,:,:]
for iter in range(0,3):  
    #subtract predictions with primary decoding dimensions (at time with max R2)
    sub_coef_array = nans([len(lag_axis),2,dim])
    sub_r2_array = nans([len(lag_axis)])

    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,weights)
        sub_r2_array[i] = r2
        sub_coef_array[i,:,:] = coef

    plt.plot(lag_axis,sub_r2_array)
    plt.title('R2 score projecting out #'+ str(iter+1) +' t_max dim')
    idx_max = np.argmax(sub_r2_array)
    time_max = lag_axis[idx_max]
    plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
    plt.legend()
    plt.xlabel('Time lag (ms)')
    plt.ylabel('R2')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'.png', dpi = 'figure')
    plt.close()


    _, _, vel_df = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field,weights)
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel('x-vel')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'_pred.png', dpi = 'figure')
    plt.close()

    #stack the decoding dimensions to be projected out
    weights = np.vstack((weights,sub_coef_array[idx_max,:,:]))

R2: 0.049721302436977144
R2: 0.06710173035639588
R2: 0.08830056259344021
R2: 0.11395545256097306
R2: 0.14448320279799942
R2: 0.17992094896974964
R2: 0.21991979701224562
R2: 0.26379106171180267
R2: 0.31034673900508014
R2: 0.3578054020478617
R2: 0.4039630156554378
R2: 0.44644547482717867
R2: 0.4831955200862339
R2: 0.5129089866623612
R2: 0.5350828266490762
R2: 0.5498056811917367
R2: 0.5574449267245716
R2: 0.5585921894959761
R2: 0.5541636658276785
R2: 0.545274160075264
R2: 0.5332659911994232
R2: 0.5198761580733626
R2: 0.5069546196501644
R2: 0.4957508646578034
R2: 0.4863644741928337
R2: 0.4778790836030775
R2: 0.4690996234219509
R2: 0.45925105242814324
R2: 0.44817206999663284
R2: 0.4361442082298802
R2: 0.5585921894959761
R2: 0.039826988015280174
R2: 0.051375449704841425
R2: 0.06412382994320776
R2: 0.07815637267468634
R2: 0.09345634070606024
R2: 0.10980094149035935
R2: 0.1266884042217068
R2: 0.14334528818976566
R2: 0.15874183073820458
R2: 0.17173587356369657
R2: 0.18129007199864378
R2: 0.1866

In [14]:
pred_range = (0, 120)
label = '_early_'
x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)

curr_r2_array = nans([len(lag_axis)])
curr_coef_array = nans([len(lag_axis),2,dim])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    curr_r2_array[i] = r2
    curr_coef_array[i,:,:] = coef

idx_max = np.argmax(curr_r2_array)
time_max = lag_axis[idx_max]
_, _, vel_df = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('x-vel')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'_pred.png', dpi = 'figure')
plt.close()

plt.plot(lag_axis, curr_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting hand velocity [0,120]')
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'.png', dpi = 'figure')
plt.close()

weights = curr_coef_array[idx_max,:,:]
for iter in range(0,3):  
    #subtract predictions with primary decoding dimensions (at time with max R2)
    sub_coef_array = nans([len(lag_axis),2,dim])
    sub_r2_array = nans([len(lag_axis)])

    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,weights)
        sub_r2_array[i] = r2
        sub_coef_array[i,:,:] = coef

    plt.plot(lag_axis,sub_r2_array)
    plt.title('R2 score projecting out #'+ str(iter+1) +' t_max dim')
    idx_max = np.argmax(sub_r2_array)
    time_max = lag_axis[idx_max]
    plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
    plt.legend()
    plt.xlabel('Time lag (ms)')
    plt.ylabel('R2')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'.png', dpi = 'figure')
    plt.close()


    _, _, vel_df = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field,weights)
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel('x-vel')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'_pred.png', dpi = 'figure')
    plt.close()

    #stack the decoding dimensions to be projected out
    weights = np.vstack((weights,sub_coef_array[idx_max,:,:]))

R2: -0.0447749909065811
R2: -0.049479313498691946
R2: -0.04894865493423439
R2: -0.04267213277405024
R2: -0.03558467071344551
R2: -0.032099802798039034
R2: -0.029954067556268393
R2: -0.020272993314270016
R2: 0.01666658081516481
R2: 0.09959913236834372
R2: 0.2183903688070863
R2: 0.3407067306166158
R2: 0.43623037137125753
R2: 0.49326341795241513
R2: 0.5166885422345626
R2: 0.5186901830962132
R2: 0.5167876850692164
R2: 0.5216712853647987
R2: 0.5314529745214682
R2: 0.5400841494630995
R2: 0.5462790984699961
R2: 0.5506414364632507
R2: 0.5510140828776319
R2: 0.5436461769241279
R2: 0.529930095017792
R2: 0.5178149350448992
R2: 0.5123670299911232
R2: 0.5122172035944931
R2: 0.513184808679229
R2: 0.5090261134271288
R2: 0.5510140828776319
R2: -0.03960815673914997
R2: -0.046045417268753
R2: -0.04531890237824365
R2: -0.038688462411687885
R2: -0.03201577020247193
R2: -0.028779821981347986
R2: -0.02673727542759541
R2: -0.0208318172365054
R2: 0.0014480333938419188
R2: 0.060695410449311704
R2: 0.1612375254

In [15]:
pred_range = (380, 500)
label = '_late_'
x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)

curr_r2_array = nans([len(lag_axis)])
curr_coef_array = nans([len(lag_axis),2,dim])
for i in range(len(lag_axis)):
    lag = lag_axis[i]
    r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
    curr_r2_array[i] = r2
    curr_coef_array[i,:,:] = coef

idx_max = np.argmax(curr_r2_array)
time_max = lag_axis[idx_max]
_, _, vel_df = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field)
for trial_dir, color in zip(plot_dir, colors):
    cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
    for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
        plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
plt.xlabel('Time (ms)')
plt.ylabel('x-vel')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'_pred.png', dpi = 'figure')
plt.close()

plt.plot(lag_axis, curr_r2_array)
plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
plt.legend()
plt.title('R2 score predicting hand velocity [380,500]')
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')
plt.tight_layout()
plt.savefig(figDir + monkey + label + str(0) +'.png', dpi = 'figure')
plt.close()

weights = curr_coef_array[idx_max,:,:]
for iter in range(0,3):  
    #subtract predictions with primary decoding dimensions (at time with max R2)
    sub_coef_array = nans([len(lag_axis),2,dim])
    sub_r2_array = nans([len(lag_axis)])

    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,weights)
        sub_r2_array[i] = r2
        sub_coef_array[i,:,:] = coef

    plt.plot(lag_axis,sub_r2_array)
    plt.title('R2 score projecting out #'+ str(iter+1) +' t_max dim')
    idx_max = np.argmax(sub_r2_array)
    time_max = lag_axis[idx_max]
    plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
    plt.legend()
    plt.xlabel('Time lag (ms)')
    plt.ylabel('R2')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'.png', dpi = 'figure')
    plt.close()


    _, _, vel_df = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field,weights)
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel('x-vel')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(iter+1) +'_pred.png', dpi = 'figure')
    plt.close()
    
    #stack the decoding dimensions to be projected out
    weights = np.vstack((weights,sub_coef_array[idx_max,:,:]))

R2: 0.3743074247389314
R2: 0.39065077153077876
R2: 0.3992724695041473
R2: 0.40527641212731536
R2: 0.4142007914070239
R2: 0.4276085669299772
R2: 0.4465612145654553
R2: 0.47120323608011194
R2: 0.4978381976733761
R2: 0.5253977099700073
R2: 0.5536517488075579
R2: 0.5781165878512429
R2: 0.5974002308627047
R2: 0.6154846163322143
R2: 0.626778504844836
R2: 0.6318249520376016
R2: 0.631156063276104
R2: 0.6249032537917194
R2: 0.6136048286660658
R2: 0.5984721127934545
R2: 0.5811012203835997
R2: 0.5620762535726369
R2: 0.5416276788702836
R2: 0.5200697547950628
R2: 0.4969627895163815
R2: 0.4711089257679689
R2: 0.44373629521704705
R2: 0.4185226956308735
R2: 0.3967669806480605
R2: 0.3788567863931591
R2: 0.6318249520376016
R2: 0.34029470616957436
R2: 0.33601491493336033
R2: 0.3192483785990856
R2: 0.2938375922375239
R2: 0.26926391540053796
R2: 0.2541986515122463
R2: 0.24447383581949722
R2: 0.2414792440639365
R2: 0.2379746128359772
R2: 0.19229314200650383
R2: 0.09864975222452144
R2: 0.026835442846740776
R

# Multi Lags

In [16]:
dataset_50ms = NWBDataset(filename, split_heldout=False)
dataset_50ms.resample(50)
print(dataset_50ms.bin_width)

50


In [17]:
n_dims = 20 # for PCA

active_mask = (~dataset_50ms.trial_info.ctr_hold_bump) & (dataset_50ms.trial_info.split != 'none')
passive_mask = (dataset_50ms.trial_info.ctr_hold_bump) & (dataset_50ms.trial_info.split != 'none')


trial_mask = active_mask
n_trials = dataset_50ms.trial_info.loc[trial_mask].shape[0]
print(n_trials,'trials')
n_neurons = dataset_50ms.data.spikes.shape[1]
print(n_neurons,'neurons')

all_data = np.array(dataset_50ms.data.spikes)
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)

scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims)
X = pca.fit(X)

PCA_data = nans([all_data.shape[0],n_dims])
idx = 0
for dp in all_data:
    dp = dp.reshape((1, -1))
    if np.isnan(dp).any():
        dp_pca = nans([1,n_dims])
    else:
        dp_pca = pca.transform(scaler.transform(dp))
    PCA_data[idx,:] = dp_pca
    idx+=1
print(PCA_data.shape)
dataset_50ms.add_continuous_data(PCA_data,'PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

371 trials
87 neurons
(53899, 87)
(53899, 87)
(53899, 20)
PCA total var explained: 0.432188868413881


In [18]:
active_data = dataset_50ms.make_trial_data(align_field='move_onset_time', align_range=(-300, 700), ignored_trials=~trial_mask)
for idx, trial in active_data.groupby('trial_id'):
    n_timepoints = trial.shape[0]
    break
print(n_timepoints,'time bins')

active_trials_neuron = nans([n_trials,n_timepoints,n_neurons])
active_trials_vel = nans([n_trials,n_timepoints,2])
active_trials_pca = nans([n_trials,n_timepoints,n_dims])
i = 0
for idx, trial in active_data.groupby('trial_id'):
    active_trials_neuron[i,:,:]=trial.spikes.to_numpy()
    active_trials_vel[i,:,:]=trial.hand_vel.to_numpy()
    active_trials_pca[i,:,:]=trial.PCA.to_numpy()
    i+=1
print(active_trials_neuron.shape)
print(active_trials_vel.shape)
print(active_trials_pca.shape)

#make dictionary for trial condition (reaching directions) for Stratified CV
active_trials_idx = np.array(dataset_50ms.trial_info.loc[trial_mask]['trial_id'])
cond_dir_idx = []
cond_dict = nans([n_trials])
for direction in [0,45,90,135,180,225,270,315]:
    cond_dir_idx.append(np.where((dataset_50ms.trial_info['cond_dir'] == direction) & (dataset_50ms.trial_info['ctr_hold_bump'] == False) & \
           (dataset_50ms.trial_info['split'] != 'none'))[0])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(cond_dir_idx)):
        if idx in cond_dir_idx[cond]:
            cond_dict[i] = cond
            break
    i+=1
print(cond_dict)

20 time bins
(371, 20, 87)
(371, 20, 2)
(371, 20, 20)
[0. 6. 6. 4. 6. 2. 0. 4. 6. 4. 6. 0. 4. 4. 4. 4. 4. 6. 4. 0. 6. 6. 0. 6.
 6. 2. 4. 6. 0. 2. 6. 4. 4. 0. 6. 6. 6. 0. 0. 2. 2. 6. 4. 4. 0. 6. 6. 6.
 6. 6. 2. 6. 6. 0. 2. 2. 2. 4. 6. 0. 2. 0. 0. 2. 0. 6. 2. 0. 0. 4. 6. 2.
 6. 6. 2. 0. 0. 6. 0. 6. 0. 4. 0. 4. 0. 4. 6. 6. 6. 0. 0. 6. 2. 0. 4. 0.
 4. 0. 2. 2. 2. 2. 4. 2. 0. 0. 0. 0. 6. 6. 0. 0. 4. 2. 6. 4. 0. 6. 2. 6.
 4. 0. 2. 4. 4. 4. 6. 4. 4. 0. 2. 4. 2. 2. 0. 0. 2. 4. 2. 2. 4. 0. 6. 2.
 4. 6. 6. 6. 4. 4. 6. 2. 6. 4. 6. 2. 4. 6. 2. 4. 6. 2. 0. 2. 6. 6. 2. 4.
 6. 6. 2. 6. 4. 6. 4. 2. 6. 0. 0. 0. 2. 0. 4. 0. 4. 0. 2. 4. 4. 2. 4. 2.
 4. 4. 6. 6. 4. 0. 4. 4. 4. 0. 0. 4. 0. 6. 4. 0. 2. 0. 0. 2. 0. 4. 0. 4.
 2. 2. 2. 2. 4. 0. 4. 4. 6. 4. 0. 4. 4. 0. 0. 6. 4. 2. 0. 4. 6. 6. 4. 0.
 6. 4. 4. 6. 2. 0. 2. 0. 2. 6. 2. 6. 2. 2. 0. 0. 4. 2. 4. 2. 2. 6. 0. 0.
 6. 6. 0. 0. 0. 0. 2. 2. 0. 0. 6. 0. 6. 4. 6. 6. 0. 6. 4. 2. 4. 2. 6. 2.
 6. 6. 0. 4. 6. 0. 0. 2. 0. 0. 4. 0. 4. 6. 4. 6. 2. 4. 6. 6. 6. 2. 6. 

## Early

### with Neurons

In [19]:
data_range = [-300,700]
pred_start = 0
pred_end = 120

idx1 = int((pred_start - data_range[0])/dataset_50ms.bin_width)
idx2 = int(n_timepoints - (data_range[1]-pred_end)/dataset_50ms.bin_width)

t_before_range = range(0,301,50);
t_after_range = range(0,501,50);

early_multi_R2s = nans([len(t_before_range),len(t_after_range)])
early_multi_coefs = []
j,k=0,0
for time_before in t_before_range:
    coef_arr = []
    for time_after in t_after_range:
        print('Predicting with',-time_before, 'to', time_after,'ms neural data')
        
        bins_before= int(time_before/dataset_50ms.bin_width) #How many bins of neural data prior to the output are used for decoding
        bins_current= 1 #Whether to use concurrent time bin of neural data
        bins_after= int(time_after/dataset_50ms.bin_width) #How many bins of neural data after the output are used for decoding

        n_total_bins = bins_before + bins_current + bins_after

        X =  nans([n_trials,idx2-idx1,n_total_bins*n_neurons])
        i = 0
        for trial_data in active_trials_neuron:
            trial_hist=get_spikes_with_history(trial_data,bins_before,bins_after,bins_current)
            trial_hist = trial_hist[idx1:idx2,:,:]
            trial_hist_flat=trial_hist.reshape(trial_hist.shape[0],(trial_hist.shape[1]*trial_hist.shape[2]))
            X[i,:,:] = trial_hist_flat
            i+=1
        
        y = active_trials_vel[:,idx1:idx2,:]
    
        lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)})
        X_reshaped = X.reshape((X.shape[0]*X.shape[1]),X.shape[2])
        y_reshaped = y.reshape((y.shape[0]*y.shape[1]),y.shape[2])
        lr_all.fit(X_reshaped, y_reshaped)
        print(lr_all.best_params_['alpha'])
        
        skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
        true_concat = nans([(n_trials*(idx2-idx1)),2])
        pred_concat = nans([(n_trials*(idx2-idx1)),2])
        trial_save_idx = 0
        for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
            #split training and testing by trials
            X_train, X_test, y_train, y_test = process_train_test(X,y,training_set,test_set)
            lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)}) 
            lr.fit(X_train, y_train)
            y_test_predicted = lr.predict(X_test)
            n = y_test_predicted.shape[0]
            true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
            pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
            trial_save_idx += n

        sses =get_sses_pred(true_concat,pred_concat)
        sses_mean=get_sses_mean(true_concat)
        early_multi_R2s[j,k] =1-np.sum(sses)/np.sum(sses_mean)     
        print('R2:',early_multi_R2s[j,k])
        coef_arr.append(lr_all.best_estimator_.coef_)
        k += 1
    j += 1
    k = 0
    early_multi_coefs.append(coef_arr)

Predicting with 0 to 0 ms neural data
100.0
R2: 0.41422941008821457
Predicting with 0 to 50 ms neural data
100.0
R2: 0.5128502743183188
Predicting with 0 to 100 ms neural data
100.0
R2: 0.5706362975605955
Predicting with 0 to 150 ms neural data
100.0
R2: 0.6112927846557585
Predicting with 0 to 200 ms neural data
100.0
R2: 0.6351267659050517
Predicting with 0 to 250 ms neural data
100.0
R2: 0.6508872052660535
Predicting with 0 to 300 ms neural data
100.0
R2: 0.6582177965889632
Predicting with 0 to 350 ms neural data
100.0
R2: 0.6674650542753541
Predicting with 0 to 400 ms neural data
100.0
R2: 0.6768030001971175
Predicting with 0 to 450 ms neural data
1000.0
R2: 0.6761994776019549
Predicting with 0 to 500 ms neural data
1000.0
R2: 0.6777298786119517
Predicting with -50 to 0 ms neural data
100.0
R2: 0.5152347979845865
Predicting with -50 to 50 ms neural data
100.0
R2: 0.5926599942735644
Predicting with -50 to 100 ms neural data
100.0
R2: 0.6380333158311494
Predicting with -50 to 150 ms n

In [20]:
fig, ax = plt.subplots()
im = ax.imshow(early_multi_R2s)
ax.set_xlabel('Length of lagging info')
ax.set_ylabel('Length of leading info')

ax.set_xticks(np.arange(len(t_after_range)))
ax.set_yticks(np.arange(len(t_before_range)))
ax.set_xticklabels(labels=t_after_range)
ax.set_yticklabels(labels=t_before_range)

ax.set_title("R2 predicting [0, 120] velocity \nwith different lagging/leading info")
fig.tight_layout()
 
for i in range(len(t_before_range)):
    for j in range(len(t_after_range)):
        text = ax.text(j, i, str(int(early_multi_R2s[i, j]*1000)/1000),
                       ha="center", va="center", color="w")

plt.tight_layout()
figDir = "/Users/sherryan/area2_population_analysis/figures/neurons/act/"
plt.savefig(figDir + monkey + '_multi_early.png', dpi = 'figure')
plt.close()

### with PCA

In [21]:
PCA_early_multi_R2s = nans([len(t_before_range),len(t_after_range)])
PCA_early_multi_coefs = []
j,k=0,0
for time_before in t_before_range:
    coef_arr = []
    for time_after in t_after_range:
        print('Predicting with',-time_before, 'to', time_after,'ms neural data')
        
        bins_before= int(time_before/dataset_50ms.bin_width) #How many bins of neural data prior to the output are used for decoding
        bins_current= 1 #Whether to use concurrent time bin of neural data
        bins_after= int(time_after/dataset_50ms.bin_width) #How many bins of neural data after the output are used for decoding

        n_total_bins = bins_before + bins_current + bins_after

        X =  nans([n_trials,idx2-idx1,n_total_bins*n_dims])
        i = 0
        for trial_data in active_trials_pca:
            trial_hist=get_spikes_with_history(trial_data,bins_before,bins_after,bins_current)
            trial_hist = trial_hist[idx1:idx2,:,:]
            trial_hist_flat=trial_hist.reshape(trial_hist.shape[0],(trial_hist.shape[1]*trial_hist.shape[2]))
            X[i,:,:] = trial_hist_flat
            i+=1
        
        y = active_trials_vel[:,idx1:idx2,:]
    
        lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)})
        X_reshaped = X.reshape((X.shape[0]*X.shape[1]),X.shape[2])
        y_reshaped = y.reshape((y.shape[0]*y.shape[1]),y.shape[2])
        lr_all.fit(X_reshaped, y_reshaped)
        print(lr_all.best_params_['alpha'])
        
        skf =StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
        true_concat = nans([(n_trials*(idx2-idx1)),2])
        pred_concat = nans([(n_trials*(idx2-idx1)),2])
        trial_save_idx = 0
        for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
            #split training and testing by trials
            X_train, X_test, y_train, y_test = process_train_test(X,y,training_set,test_set)
            lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)}) 
            lr.fit(X_train, y_train)
            y_test_predicted = lr.predict(X_test)
            n = y_test_predicted.shape[0]
            true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
            pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
            trial_save_idx += n

        sses =get_sses_pred(true_concat,pred_concat)
        sses_mean=get_sses_mean(true_concat)
        PCA_early_multi_R2s[j,k] =1-np.sum(sses)/np.sum(sses_mean)     
        print('R2:',PCA_early_multi_R2s[j,k])
        coef_arr.append(lr_all.best_estimator_.coef_)
        k += 1
    j += 1
    k = 0
    PCA_early_multi_coefs.append(coef_arr)

Predicting with 0 to 0 ms neural data
100.0
R2: 0.33376730533028376
Predicting with 0 to 50 ms neural data
100.0
R2: 0.43247328959568654
Predicting with 0 to 100 ms neural data
100.0
R2: 0.48750461231304376
Predicting with 0 to 150 ms neural data
1000.0
R2: 0.5316391028617087
Predicting with 0 to 200 ms neural data
1000.0
R2: 0.5549895374519626
Predicting with 0 to 250 ms neural data
1000.0
R2: 0.5813314340720162
Predicting with 0 to 300 ms neural data
1000.0
R2: 0.598130245356703
Predicting with 0 to 350 ms neural data
1000.0
R2: 0.6191225301639743
Predicting with 0 to 400 ms neural data
1000.0
R2: 0.6276581509045009
Predicting with 0 to 450 ms neural data
1000.0
R2: 0.6342110488611694
Predicting with 0 to 500 ms neural data
1000.0
R2: 0.6368165492745863
Predicting with -50 to 0 ms neural data
100.0
R2: 0.4617229349565739
Predicting with -50 to 50 ms neural data
100.0
R2: 0.5172985341628616
Predicting with -50 to 100 ms neural data
1000.0
R2: 0.5556314694904743
Predicting with -50 to 

In [22]:
fig, ax = plt.subplots()
im = ax.imshow(PCA_early_multi_R2s)
ax.set_xlabel('Length of lagging info')
ax.set_ylabel('Length of leading info')

ax.set_xticks(np.arange(len(t_after_range)))
ax.set_yticks(np.arange(len(t_before_range)))
ax.set_xticklabels(labels=t_after_range)
ax.set_yticklabels(labels=t_before_range)

ax.set_title("R2 predicting [0, 120] velocity \nwith different lagging/leading info")
fig.tight_layout()

for i in range(len(t_before_range)):
    for j in range(len(t_after_range)):
        text = ax.text(j, i, str(int(PCA_early_multi_R2s[i, j]*1000)/1000),
                       ha="center", va="center", color="w")
plt.tight_layout()
figDir = "/Users/sherryan/area2_population_analysis/figures/PCA/act/"
plt.savefig(figDir + monkey + '_multi_early.png', dpi = 'figure')
plt.close()

## Whole

### with Neurons

In [23]:
data_range = [-300,700]
pred_start = -100
pred_end = 500

idx1 = int((pred_start - data_range[0])/dataset_50ms.bin_width)
idx2 = int(n_timepoints - (data_range[1]-pred_end)/dataset_50ms.bin_width)

t_before_range = range(0,201,50);
t_after_range = range(0,201,50);

whole_multi_R2s = nans([len(t_before_range),len(t_after_range)])
whole_multi_coefs = []
j,k=0,0
for time_before in t_before_range:
    coef_arr = []
    for time_after in t_after_range:
        print('Predicting with',-time_before, 'to', time_after,'ms neural data')
        
        bins_before= int(time_before/dataset_50ms.bin_width) #How many bins of neural data prior to the output are used for decoding
        bins_current= 1 #Whether to use concurrent time bin of neural data
        bins_after= int(time_after/dataset_50ms.bin_width) #How many bins of neural data after the output are used for decoding

        n_total_bins = bins_before + bins_current + bins_after

        X =  nans([n_trials,idx2-idx1,n_total_bins*n_neurons])
        i = 0
        for trial_data in active_trials_neuron:
            trial_hist=get_spikes_with_history(trial_data,bins_before,bins_after,bins_current)
            trial_hist = trial_hist[idx1:idx2,:,:]
            trial_hist_flat=trial_hist.reshape(trial_hist.shape[0],(trial_hist.shape[1]*trial_hist.shape[2]))
            X[i,:,:] = trial_hist_flat
            i+=1
        
        y = active_trials_vel[:,idx1:idx2,:]
    
        lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)})
        X_reshaped = X.reshape((X.shape[0]*X.shape[1]),X.shape[2])
        y_reshaped = y.reshape((y.shape[0]*y.shape[1]),y.shape[2])
        lr_all.fit(X_reshaped, y_reshaped)
        print(lr_all.best_params_['alpha'])
        
        skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
        true_concat = nans([(n_trials*(idx2-idx1)),2])
        pred_concat = nans([(n_trials*(idx2-idx1)),2])
        trial_save_idx = 0
        for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
            #split training and testing by trials
            X_train, X_test, y_train, y_test = process_train_test(X,y,training_set,test_set)
            lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)}) 
            lr.fit(X_train, y_train)
            y_test_predicted = lr.predict(X_test)
            n = y_test_predicted.shape[0]
            true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
            pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
            trial_save_idx += n

        sses =get_sses_pred(true_concat,pred_concat)
        sses_mean=get_sses_mean(true_concat)
        whole_multi_R2s[j,k] =1-np.sum(sses)/np.sum(sses_mean)     
        print('R2:',whole_multi_R2s[j,k])
        coef_arr.append(lr_all.best_estimator_.coef_)
        k += 1
    j += 1
    k = 0
    whole_multi_coefs.append(coef_arr)

Predicting with 0 to 0 ms neural data
100.0
R2: 0.467736481437361
Predicting with 0 to 50 ms neural data
100.0
R2: 0.5712959296734621
Predicting with 0 to 100 ms neural data
100.0
R2: 0.6161304320153006
Predicting with 0 to 150 ms neural data
100.0
R2: 0.6417303565394066
Predicting with 0 to 200 ms neural data
100.0
R2: 0.6604685426737124
Predicting with -50 to 0 ms neural data
100.0
R2: 0.5731280728733198
Predicting with -50 to 50 ms neural data
100.0
R2: 0.6356185299974026
Predicting with -50 to 100 ms neural data
100.0
R2: 0.6652718454871442
Predicting with -50 to 150 ms neural data
100.0
R2: 0.6838607919797849
Predicting with -50 to 200 ms neural data
100.0
R2: 0.6981663725911744
Predicting with -100 to 0 ms neural data
100.0
R2: 0.6197193518684381
Predicting with -100 to 50 ms neural data
100.0
R2: 0.6684559340938048
Predicting with -100 to 100 ms neural data
100.0
R2: 0.6927987876851771
Predicting with -100 to 150 ms neural data
100.0
R2: 0.7083032612049315
Predicting with -100 t

In [24]:
fig, ax = plt.subplots()
im = ax.imshow(whole_multi_R2s)
ax.set_xlabel('Length of lagging info')
ax.set_ylabel('Length of leading info')

ax.set_xticks(np.arange(len(t_after_range)))
ax.set_yticks(np.arange(len(t_before_range)))
ax.set_xticklabels(labels=t_after_range)
ax.set_yticklabels(labels=t_before_range)

ax.set_title("R2 predicting [-100, 500] velocity \nwith different lagging/leading info")
fig.tight_layout()

for i in range(len(t_before_range)):
    for j in range(len(t_after_range)):
        text = ax.text(j, i, str(int(whole_multi_R2s[i, j]*1000)/1000),
                       ha="center", va="center", color="w")
plt.tight_layout()
figDir = "/Users/sherryan/area2_population_analysis/figures/neurons/act/"
plt.savefig(figDir + monkey + '_multi_whole.png', dpi = 'figure')
plt.close()

In [25]:
coef_X = whole_multi_coefs[-1][-1][0] #which entry's weights to use

t_label = np.arange(-200,201,50)

n_weights = len(t_before_range) + len(t_after_range) - 1
coef_X_reshaped = coef_X.reshape(n_weights,n_neurons)
angDist_array = nans([n_weights,n_weights])
for i in range(n_weights):
    for j in range(n_weights):
        angDist_array[i,j] = math.degrees(angle_between(coef_X_reshaped[i,:],coef_X_reshaped[j,:]))
fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(angDist_array)
ax.set_xlabel('Bin time (ms)')
ax.set_ylabel('Bin time (ms)')

ax.set_xticks(np.arange(len(t_label)))
ax.set_yticks(np.arange(len(t_label)))
ax.set_xticklabels(labels=t_label)
ax.set_yticklabels(labels=t_label)

ax.set_title("Angle between weight vectors at time points")
fig.tight_layout()

for i in range(len(t_label)):
    for j in range(len(t_label)):
        text = ax.text(j, i, str(int(angDist_array[i, j])),
                       ha="center", va="center", color="w")
plt.tight_layout()
figDir = "/Users/sherryan/area2_population_analysis/figures/neurons/act/"
plt.savefig(figDir + monkey + '_multi_whole_deg.png', dpi = 'figure')
plt.close()

### with PCA

In [26]:
PCA_whole_multi_R2s = nans([len(t_before_range),len(t_after_range)])
PCA_whole_multi_coefs = []
j,k=0,0
for time_before in t_before_range:
    coef_arr = []
    for time_after in t_after_range:
        print('Predicting with',-time_before, 'to', time_after,'ms neural data')
        
        bins_before= int(time_before/dataset_50ms.bin_width) #How many bins of neural data prior to the output are used for decoding
        bins_current= 1 #Whether to use concurrent time bin of neural data
        bins_after= int(time_after/dataset_50ms.bin_width) #How many bins of neural data after the output are used for decoding

        n_total_bins = bins_before + bins_current + bins_after

        X =  nans([n_trials,idx2-idx1,n_total_bins*n_dims])
        i = 0
        for trial_data in active_trials_pca:
            trial_hist=get_spikes_with_history(trial_data,bins_before,bins_after,bins_current)
            trial_hist = trial_hist[idx1:idx2,:,:]
            trial_hist_flat=trial_hist.reshape(trial_hist.shape[0],(trial_hist.shape[1]*trial_hist.shape[2]))
            X[i,:,:] = trial_hist_flat
            i+=1
        
        y = active_trials_vel[:,idx1:idx2,:]
    
        lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)})
        X_reshaped = X.reshape((X.shape[0]*X.shape[1]),X.shape[2])
        y_reshaped = y.reshape((y.shape[0]*y.shape[1]),y.shape[2])
        lr_all.fit(X_reshaped, y_reshaped)
        print(lr_all.best_params_['alpha'])
        
        skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
        true_concat = nans([(n_trials*(idx2-idx1)),2])
        pred_concat = nans([(n_trials*(idx2-idx1)),2])
        trial_save_idx = 0
        for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
            #split training and testing by trials
            X_train, X_test, y_train, y_test = process_train_test(X,y,training_set,test_set)
            lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)}) 
            lr.fit(X_train, y_train)
            y_test_predicted = lr.predict(X_test)
            n = y_test_predicted.shape[0]
            true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
            pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
            trial_save_idx += n

        sses =get_sses_pred(true_concat,pred_concat)
        sses_mean=get_sses_mean(true_concat)
        PCA_whole_multi_R2s[j,k] =1-np.sum(sses)/np.sum(sses_mean)     
        print('R2:',PCA_whole_multi_R2s[j,k])
        coef_arr.append(lr_all.best_estimator_.coef_)
        k += 1
    j += 1
    k = 0
    PCA_whole_multi_coefs.append(coef_arr)

Predicting with 0 to 0 ms neural data
100.0
R2: 0.3924697900837686
Predicting with 0 to 50 ms neural data
100.0
R2: 0.5028486848661706
Predicting with 0 to 100 ms neural data
100.0
R2: 0.5566503525764448
Predicting with 0 to 150 ms neural data
1000.0
R2: 0.5901526791834324
Predicting with 0 to 200 ms neural data
1000.0
R2: 0.6125905022879805
Predicting with -50 to 0 ms neural data
100.0
R2: 0.48620053961584575
Predicting with -50 to 50 ms neural data
100.0
R2: 0.557026892683411
Predicting with -50 to 100 ms neural data
1000.0
R2: 0.59839707680401
Predicting with -50 to 150 ms neural data
1000.0
R2: 0.6248358338634898
Predicting with -50 to 200 ms neural data
1000.0
R2: 0.6444327421856828
Predicting with -100 to 0 ms neural data
1000.0
R2: 0.5222994275344095
Predicting with -100 to 50 ms neural data
1000.0
R2: 0.581890966992592
Predicting with -100 to 100 ms neural data
1000.0
R2: 0.6169814699339284
Predicting with -100 to 150 ms neural data
1000.0
R2: 0.6421739286416924
Predicting with

In [27]:
fig, ax = plt.subplots()
im = ax.imshow(PCA_whole_multi_R2s)
ax.set_xlabel('Length of lagging info')
ax.set_ylabel('Length of leading info')

ax.set_xticks(np.arange(len(t_after_range)))
ax.set_yticks(np.arange(len(t_before_range)))
ax.set_xticklabels(labels=t_after_range)
ax.set_yticklabels(labels=t_before_range)

ax.set_title("R2 predicting [-100, 500] velocity \nwith different lagging/leading info")
fig.tight_layout()

for i in range(len(t_before_range)):
    for j in range(len(t_after_range)):
        text = ax.text(j, i, str(int(PCA_whole_multi_R2s[i, j]*1000)/1000),
                       ha="center", va="center", color="w")
plt.tight_layout()
figDir = "/Users/sherryan/area2_population_analysis/figures/PCA/act/"
plt.savefig(figDir + monkey + '_multi_whole.png', dpi = 'figure')
plt.close()

In [28]:
coef_X = PCA_whole_multi_coefs[-1][-1][0] #which entry's weights to use

t_label = np.arange(-200,201,50)

n_weights = len(t_before_range) + len(t_after_range) - 1
coef_X_reshaped = coef_X.reshape(n_weights,n_dims)
angDist_array = nans([n_weights,n_weights])
for i in range(n_weights):
    for j in range(n_weights):
        angDist_array[i,j] = math.degrees(angle_between(coef_X_reshaped[i,:],coef_X_reshaped[j,:]))
fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(angDist_array)
ax.set_xlabel('Bin time (ms)')
ax.set_ylabel('Bin time (ms)')

ax.set_xticks(np.arange(len(t_label)))
ax.set_yticks(np.arange(len(t_label)))
ax.set_xticklabels(labels=t_label)
ax.set_yticklabels(labels=t_label)

ax.set_title("Angle between weight vectors at time points")
fig.tight_layout()

for i in range(len(t_label)):
    for j in range(len(t_label)):
        text = ax.text(j, i, str(int(angDist_array[i, j])),
                       ha="center", va="center", color="w")
plt.tight_layout()
figDir = "/Users/sherryan/area2_population_analysis/figures/PCA/act/"
plt.savefig(figDir + monkey + '_multi_whole_deg.png', dpi = 'figure')
plt.close()